In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
import tempfile
from collections import defaultdict

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import custom_object_scope

In [3]:
from qkeras import *
from qkeras import QActivation
from qkeras import QDense
from qkeras import QConv2D
from qkeras import quantized_bits, quantized_relu
from qkeras.utils import load_qmodel
from qkeras.utils import print_model_sparsity
from qkeras.utils import model_save_quantized_weights

In [4]:
NB_EPOCH = 30
BATCH_SIZE = 64
VERBOSE = 1
NB_CLASSES = 10
VALIDATION_SPLIT = 0.1
OPTIMIZER = Adam(learning_rate=0.001)
#OPTIMIZER = SGD(learning_rate=0.001, momentum=0.9)

W = 4  #QConv2d q_bit
I = 0
Z = float(4)  #QActivation quantize_bit

In [15]:
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    return lr * tf.math.exp(-0.1)

CALLBACKS = lr_schedule = LearningRateScheduler(scheduler)

In [5]:
def build_model(input_shape):
    x = x_in = Input(shape=input_shape, name="input_layer")
    a = QConv2D(
        32, (2, 2), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="conv2d_L1")(x)
    b = QActivation("quantized_relu(4, 0)", name="activation_1")(a)
    c = QConv2D(
        64, (3, 3), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="conv2d_L2")(b)
    d = QActivation("quantized_relu(4, 0)", name="activation_2")(c)
    e = QConv2D(
        64, (2, 2), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="conv2d_L3")(d)
    f = QActivation("quantized_relu(4, 0)", name="activation_3")(e)
    g = Flatten()(f)
    h = QDense(NB_CLASSES, kernel_quantizer=quantized_bits(W, I, 1),
                   bias_quantizer=quantized_bits(W, I, 1),
                   name="dense")(g)
    i = Activation("softmax", name="Softmax")(h)

    model = Model(inputs=[x_in], outputs=[i])
    return model

In [16]:
def train_and_save(model, x_train, y_train, x_test, y_test):
    model.compile(
        loss="categorical_crossentropy",
     #   loss="sparse_categorical_crossentropy",
     #   loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer= OPTIMIZER,
        metrics=["accuracy"])
    
    # Print the model summary.
    model.summary()
    
  #  callbacks=[
   #     gradual_qnoise_callback_4
    #]
    
    model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=NB_EPOCH,
        validation_split=VALIDATION_SPLIT,
        verbose=VERBOSE,
        callbacks=[CALLBACKS],
        validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    print_model_sparsity(model)
    
    # Export and import the model. Check that accuracy persists.
    _, keras_file = tempfile.mkstemp(suffix=".keras")  # Ensure the suffix is .keras
    print("Saving model to:", keras_file)
    # Save the model in the new .keras format
    #save_model(model, keras_file)
    model.save("modelx.keras", save_format="keras")
    print("Model saved to modelx.keras")
    print_qstats(model)

In [7]:
data = np.load('/home/mindspore/work/mnist.npz')
x_train, y_train = data['x_train'], data['y_train']
x_test, y_test = data['x_test'], data['y_test']

In [8]:
img_rows, img_cols = 28, 28

if K.image_data_format() == "channels_first":
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [9]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices //one-hot key 0-9 class matrice lable 0/1
y_train = to_categorical(y_train, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:

model = build_model(input_shape)


In [ ]:

#train model and save data

train_and_save(model, x_train, y_train, x_test, y_test)



In [ ]:
print("Reloading model")
    
with custom_object_scope({'TFOpLambda': Lambda}):
        loaded_model = load_qmodel('modelx.keras')

loaded_model.summary()  

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])